In [209]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

mysteamid=76561198249806082

In [ ]:
with open('Steam Community Counter-Strike Global Offensive Persönliche Spieldaten.html', 'rb') as html:
    soup = BeautifulSoup(html)

In [18]:
with open('small.htm', 'rb') as html:
    soup = BeautifulSoup(html)

In [20]:
print(soup.prettify())
"csgo_scoreboard_inner_left"

<!DOCTYPE html>
<html class="responsive" lang="de">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="#171a21" name="theme-color"/>
  <title>
   Steam Community :: Counter-Strike: Global Offensive :: Persönliche Spieldaten
  </title>
  <link href="https://steamcommunity.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="small-Dateien/motiva_sans.css" rel="stylesheet" type="text/css"/>
  <link href="small-Dateien/buttons.css" rel="stylesheet" type="text/css"/>
  <link href="small-Dateien/shared_global.css" rel="stylesheet" type="text/css"/>
  <link href="small-Dateien/globalv2.css" rel="stylesheet" type="text/css"/>
  <link href="small-Dateien/personal_data_csgo.css" rel="stylesheet" type="text/css"/>
  <link href="small-Dateien/playerstats_generic.css" rel="stylesheet" type="text/css"/>
  <link href="small-Dateien/profilev2.css" rel="stylesheet" typ

In [196]:
def get_maps(soup):
    """returns a list off the pure html tables which include data regarding the map """
    return soup.findAll('table',{'class':"csgo_scoreboard_inner_left"})
def get_scoreboard(soup):
    """returns a list off the pure html tables which include data regarding the scoreboard """
    return soup.findAll('table',{'class':"csgo_scoreboard_inner_right banchecker-formatted banchecker-withcolumn"})
def get_steamids(table_scoreboard):
    steamids=[int(str(table_scoreboard.findAll('tr',{'class':"banchecker-profile banchecker-checking banchecker-checked"})[i]).split('data-steamid64="')[1].split('">')[0]) for i in range(10)]
    return steamids
def parse_scoreboard(soup):
    raw_list=get_scoreboard(soup)
    list_of_dfs_noskip=pd.read_html(str(get_scoreboard(soup)))
    scores=[df.iloc[5,0] for df in list_of_dfs_noskip]
    list_of_dfs=pd.read_html(str(get_scoreboard(soup)),skiprows=[6])
    scores=[df.iloc[5,0] for df in list_of_dfs_noskip]
    steamids_games=[get_steamids(table_scoreboard) for table_scoreboard in raw_list]
    
    for df,steamids in zip(list_of_dfs,steamids_games):
        df.insert(0,"steamid64",steamids)
    
    for score,steamids in zip(scores,steamids_games):
        position=steamids.index(mysteamid) # position of my steamid (where i am on the leaderboard)
        if position<=4: #determens if the left side of the score ( 16:11 ) is my team or not
            left=True
            
        else:
            left=False
        
    return (list_of_dfs,scores,steamids_games)

In [212]:
parse_scoreboard(soup)[1][0].split(":")


['11 ', ' 16']

In [50]:
tables_map=soup.findAll('table',{'class':"csgo_scoreboard_inner_left"})
tables_map

[<table class="csgo_scoreboard_inner_left">
 <tbody><tr>
 <td>
 						Competitive Mirage					</td>
 </tr>
 <tr>
 <td>
 						2021-03-04 13:13:08 GMT					</td>
 </tr>
 <tr>
 <td>
 						Wait Time: 00:39					</td>
 </tr>
 <tr>
 <td>
 						Match Duration: 41:09					</td>
 </tr>
 <tr>
 <td class="csgo_scoreboard_cell_noborder">
 <a href="http://replay271.valve.net/730/003467898758207898043_1101313101.dem.bz2" target="_blank">
 <div class="btnv6_blue_hoverfade btn_medium csgo_scoreboard_btn_gotv">
 									Download GOTV Replay								</div>
 </a>
 </td>
 </tr>
 </tbody></table>,
 <table class="csgo_scoreboard_inner_left">
 <tbody><tr>
 <td>
 						Competitive Overpass					</td>
 </tr>
 <tr>
 <td>
 						2021-03-03 15:39:24 GMT					</td>
 </tr>
 <tr>
 <td>
 						Wait Time: 02:17					</td>
 </tr>
 <tr>
 <td>
 						Match Duration: 39:15					</td>
 </tr>
 <tr>
 <td class="csgo_scoreboard_cell_noborder">
 <a href="http://replay188.valve.net/730/003467731795649233101_1396317311.dem.bz2" targe

In [127]:
tables_scoreboard=soup.findAll('table',{'class':"csgo_scoreboard_inner_right banchecker-formatted banchecker-withcolumn"})

steamids=[get_steamid(tables_scoreboard[0],i) for i in range(10)]
steamids
#print(tables_scoreboard[0].prettify())
#int(str(tables_scoreboard[0].findAll('tr',{'class':"banchecker-profile banchecker-checking banchecker-checked"})[4]).split('data-steamid64="')[1].split('">')[0])

[76561198353774782,
 76561198447715634,
 76561198249806082,
 76561198963747625,
 76561197979024710,
 76561198044086513,
 76561198071655140,
 76561198074100832,
 76561198143579483,
 76561198119579852]